In [1]:
!pip install psycopg2-binary | tail -n 1
!pip install ibm-ai-openscale==2.1.14 --no-cache | tail -n 1
!pip install --upgrade numpy --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1
!pip install --user -U pyspark==2.1.2 --no-cache | tail -n 1

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: numpy in /opt/conda3/lib/python3.5/site-packages (1.17.2)
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --upgrade scikit-learn==0.20.3

Requirement already up-to-date: scikit-learn==0.20.3 in /opt/conda3/lib/python3.5/site-packages (0.20.3)
You should consider upgrading via the 'pip install --upgrade pip' command.


# Restart the Kernel Now
Always restart the kernel after installing new packages

# Load and Clean Data

In [3]:
# Place cursor below and insert the Pandas DataFrame for your WA_Fn-UseC_-Telco-Customer-Churn.csv data
# Make sure the variable is named `df_data_1` for the line `df_data_1 = pd.read_csv(body)`

import dsx_core_utils, requests, jaydebeapi, os, io, sys
from pyspark.sql import SparkSession
import pandas as pd
df1 = None
dataSet = dsx_core_utils.get_remote_data_set_info('USER1066.billingProductCustomers')
dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])
if (sys.version_info >= (3, 0)):
  conn = jaydebeapi.connect(dataSource['driver_class'], dataSource['URL'], [dataSource['user'], dataSource['password']])
else:
  conn = jaydebeapi.connect(dataSource['driver_class'], [dataSource['URL'], dataSource['user'], dataSource['password']])
query = 'select * from "' + (dataSet['schema'] + '"."' if (len(dataSet['schema'].strip()) != 0) else '') +  dataSet['table'] + '"'

if (dataSet['query']):
    query = dataSet['query']
df1 = pd.read_sql(query, con=conn)
df1.head()




,Churn,Contract,Dependents,MonthlyCharges,PaperlessBilling,Partner,PaymentMethod,SeniorCitizen,TotalCharges,customerID,...,tenure,DeviceProtection,InternetService,MultipleLines,OnlineBackup,OnlineSecurity,PhoneService,StreamingMovies,StreamingTV,TechSupport
0,No,Two year,Yes,19.55,No,Yes,Bank transfer (automatic),0,1463.45,1166-PQLGG,...,72,No internet service,No,No,No internet service,No internet service,Yes,No internet service,No internet service,No internet service
1,No,One year,No,19.60,No,Yes,Bank transfer (automatic),0,189.45,3146-JTQHR,...,10,No internet service,No,No,No internet service,No internet service,Yes,No internet service,No internet service,No internet service
2,No,One year,Yes,20.05,No,No,Mailed check,0,96.80,4291-YZODP,...,7,No internet service,No,No,No internet service,No internet service,Yes,No internet service,No internet service,No internet service
3,No,Month-to-month,No,85.55,Yes,No,Electronic check,1,408.50,4395-PZMSN,...,5,No,Fiber optic,No,Yes,No,Yes,Yes,No,No
4,No,Two year,Yes,78.60,No,Yes,Credit card (automatic),0,1846.65,6427-FEFIG,...,24,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,No


In [5]:
df_data_1 = df1
# Drop customerID column
df_data_1 = df_data_1.drop('customerID', axis=1)
df_data_1.head(5)

,Churn,Contract,Dependents,MonthlyCharges,PaperlessBilling,Partner,PaymentMethod,SeniorCitizen,TotalCharges,gender,tenure,DeviceProtection,InternetService,MultipleLines,OnlineBackup,OnlineSecurity,PhoneService,StreamingMovies,StreamingTV,TechSupport
0,No,Two year,Yes,19.55,No,Yes,Bank transfer (automatic),0,1463.45,Female,72,No internet service,No,No,No internet service,No internet service,Yes,No internet service,No internet service,No internet service
1,No,One year,No,19.60,No,Yes,Bank transfer (automatic),0,189.45,Male,10,No internet service,No,No,No internet service,No internet service,Yes,No internet service,No internet service,No internet service
2,No,One year,Yes,20.05,No,No,Mailed check,0,96.80,Female,7,No internet service,No,No,No internet service,No internet service,Yes,No internet service,No internet service,No internet service
3,No,Month-to-month,No,85.55,Yes,No,Electronic check,1,408.50,Male,5,No,Fiber optic,No,Yes,No,Yes,Yes,No,No
4,No,Two year,Yes,78.60,No,Yes,Credit card (automatic),0,1846.65,Female,24,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,No


In [6]:
df_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
Churn               7043 non-null object
Contract            7043 non-null object
Dependents          7043 non-null object
MonthlyCharges      7043 non-null float64
PaperlessBilling    7043 non-null object
Partner             7043 non-null object
PaymentMethod       7043 non-null object
SeniorCitizen       7043 non-null int64
TotalCharges        7032 non-null float64
gender              7043 non-null object
tenure              7043 non-null int64
DeviceProtection    7043 non-null object
InternetService     7043 non-null object
MultipleLines       7043 non-null object
OnlineBackup        7043 non-null object
OnlineSecurity      7043 non-null object
PhoneService        7043 non-null object
StreamingMovies     7043 non-null object
StreamingTV         7043 non-null object
TechSupport         7043 non-null object
dtypes: float64(2), int64(2), object(16)
memory usage: 1.1+ MB


In [7]:
# Check if we have any NaN values
df_data_1.isnull().values.any()

True

In [8]:
# Handle missing values for column 8, TotalCharges
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values="NaN", strategy="mean")

df_data_1.iloc[:, 8] = imp.fit_transform(df_data_1.iloc[:, 8].values.reshape(-1, 1))
df_data_1.iloc[:, 8] = pd.Series(df_data_1.iloc[:, 8])

/opt/conda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
# Check if we have any NaN values
df_data_1.isnull().values.any()

False

# Create a model

In [10]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
df_data = spark.createDataFrame(df_data_1)
df_data.head()

Row(Churn='No', Contract='Two year', Dependents='Yes', MonthlyCharges=19.55, PaperlessBilling='No', Partner='Yes', PaymentMethod='Bank transfer (automatic)', SeniorCitizen=0, TotalCharges=1463.45, gender='Female', tenure=72, DeviceProtection='No internet service', InternetService='No', MultipleLines='No', OnlineBackup='No internet service', OnlineSecurity='No internet service', PhoneService='Yes', StreamingMovies='No internet service', StreamingTV='No internet service', TechSupport='No internet service')

### Modify the MODEL_NAME and DEPLOY_NAME if you are re-running this notebook, and want multiple versions

In [11]:
MODEL_NAME = "TelcoChurn-v2"
DEPLOYMENT_NAME = "TelcoChurn-v2"

In [12]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)


print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 5661
Number of records for evaluation: 1382
root
 |-- Churn: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- SeniorCitizen: long (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- tenure: long (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- TechSupport: string (nullable = true)



In [13]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model


si_gender = StringIndexer(inputCol = 'gender', outputCol = 'gender_IX')
si_Partner = StringIndexer(inputCol = 'Partner', outputCol = 'Partner_IX')
si_Dependents = StringIndexer(inputCol = 'Dependents', outputCol = 'Dependents_IX')
si_PhoneService = StringIndexer(inputCol = 'PhoneService', outputCol = 'PhoneService_IX')
si_MultipleLines = StringIndexer(inputCol = 'MultipleLines', outputCol = 'MultipleLines_IX')
si_InternetService = StringIndexer(inputCol = 'InternetService', outputCol = 'InternetService_IX')
si_OnlineSecurity = StringIndexer(inputCol = 'OnlineSecurity', outputCol = 'OnlineSecurity_IX')
si_OnlineBackup = StringIndexer(inputCol = 'OnlineBackup', outputCol = 'OnlineBackup_IX')
si_DeviceProtection = StringIndexer(inputCol = 'DeviceProtection', outputCol = 'DeviceProtection_IX')
si_TechSupport = StringIndexer(inputCol = 'TechSupport', outputCol = 'TechSupport_IX')
si_StreamingTV = StringIndexer(inputCol = 'StreamingTV', outputCol = 'StreamingTV_IX')
si_StreamingMovies = StringIndexer(inputCol = 'StreamingMovies', outputCol = 'StreamingMovies_IX')
si_Contract = StringIndexer(inputCol = 'Contract', outputCol = 'Contract_IX')
si_PaperlessBilling = StringIndexer(inputCol = 'PaperlessBilling', outputCol = 'PaperlessBilling_IX')
si_PaymentMethod = StringIndexer(inputCol = 'PaymentMethod', outputCol = 'PaymentMethod_IX')


In [14]:
si_Label = StringIndexer(inputCol="Churn", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [15]:
va_features = VectorAssembler(inputCols=['gender_IX',  'SeniorCitizen', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', \
                                         'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', \
                                         'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'TotalCharges', 'MonthlyCharges'], outputCol="features")

In [16]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_gender, si_Partner, si_Dependents, si_PhoneService, si_MultipleLines, si_InternetService, si_OnlineSecurity, si_OnlineBackup, si_DeviceProtection, \
                            si_TechSupport, si_StreamingTV, si_StreamingMovies, si_Contract, si_PaperlessBilling, si_PaymentMethod, si_Label, va_features, \
                            classifier, label_converter])

model = pipeline.fit(train_data)

In [17]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.709878


# Save and deploy the model

In [18]:
from dsx_ml.ml import save

In [19]:
save(name=MODEL_NAME,
    model=model,
    test_data = test_data,
    algorithm_type='Classification',
    description='This is a SparkML Model to Classify Telco Customer Churn Risk')

Using TensorFlow backend.


{'path': '/user-home/1066/DSX_Projects/scottda-project-9-10/models/TelcoChurn-v2/1',
 'scoring_endpoint': 'https://dsxl-api/v3/project/score/Python35/spark-2.2/scottda-project-9-10/TelcoChurn-v2/1'}

In [22]:
# Write the test data without label to a .csv so that we can later use it for batch scoring
write_score_CSV=test_data.toPandas().drop(['Churn'], axis=1)
write_score_CSV.to_csv('../datasets/TelcoCustomerSparkMLBatchScore.csv', sep=',', index=False)

In [24]:
# Write the test data to a .csv so that we can later use it for evaluation
write_eval_CSV=test_data.toPandas()
write_eval_CSV.to_csv('../datasets/TelcoCustomerSparkMLEval.csv', sep=',', index=False)